In [ ]:
from ros_edf.ros_interface import EdfRosInterface
from edf.data import PointCloud, SE3, TargetPoseDemo, DemoSequence, DemoSeqDataset, save_demos
from edf.pc_utils import optimize_pcd_collision, draw_geometry
from edf.preprocess import Rescale, NormalizeColor, Downsample, ApplySE3
from edf.agent import PickAgent, PlaceAgent

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose

torch.set_printoptions(precision= 3, sci_mode=False, linewidth=120)

# Initialize Robot Interface

In [ ]:
env_interface = EdfRosInterface(reference_frame = "scene")
env_interface.reset()
env_interface.moveit_interface.arm_group.set_planning_time(seconds=5)

In [ ]:
plan_targets = [(SE3([0., 0., 1., 0., 0., 0., 0.6]), "default", {}),
                (SE3([0., 0., 1., 0., 0., 0., 0.4]), "default", {}),
                (SE3([0., 0., 1., 0., 0., 0., 0.6]), "default", {}),
                (SE3([[0., 0., 1., 0., 0., 0., 0.4],[0., 0., 1., 0., 0., 0., 0.6]]), "cartesian", {'cartesian_step': 0.01, 'cspace_step_thr': 10., 'avoid_collision': False, 'success_fraction': 0.95}),
                (SE3([[0., 0., 1., 0., 0., 0., 0.4],[0., 0., 1., 0., 0., 0., 0.6]]), "cartesian", {'cartesian_step': 0.01, 'cspace_step_thr': 10., 'avoid_collision': False, 'success_fraction': 0.95}),
                (SE3([0., 0., 1., 0., 0., 0., 0.5]), "default", {}),
               ]


plan_results, plans = env_interface.move_plans(targets=plan_targets)

In [ ]:
if plan_results[-1] is True:
    env_interface.execute_plans(plans)